In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)

In [2]:
import pandas as pd
import subprocess

In [3]:
import os
clear = lambda: os.system('clear')

In [4]:
def instances_group(data):
    indices = [index for index, value in enumerate(data) if value == "TERMINATED" or value == "RUNNING"]
    
    start_index = 0
    end_index = 0   
    
    chunks = []

    for index, index_value in enumerate(indices):
        end_index = index_value + 1
        
        instances = data[start_index:end_index]
        instances.insert(3, False)
        
        if len(instances) == 6:
            instances.insert(-1, "UNASSIGNED")
        
        chunks.append(instances)
        start_index = end_index
        
    return chunks

In [5]:
def get_instances():
    cmd = "gcloud compute instances list"
    
    output = subprocess.check_output(cmd, shell=True)
    output = output.decode("utf-8")
    output = output.split()
    
    columns = output[:7]
    data = output[7:]
    
    chunks = instances_group(data)
    instances = pd.DataFrame(chunks, columns=columns)
    
    return instances

In [6]:
instances = get_instances()
instances

,NAME,ZONE,MACHINE_TYPE,PREEMPTIBLE,INTERNAL_IP,EXTERNAL_IP,STATUS
0,fastai-k80-vm,asia-east1-b,n1-standard-4,False,10.140.0.2,UNASSIGNED,TERMINATED
1,fastai-courses,us-east1-c,n1-standard-4,False,10.142.0.2,UNASSIGNED,TERMINATED
2,fastai-micro-vm,us-west1-b,n1-standard-1,False,10.138.0.2,UNASSIGNED,TERMINATED


In [ ]:
def choose_instance(instance_state):
    if instance_state == "start":
        instances_names = instances[instances["STATUS"] == "TERMINATED"]
    else:
        instances_names = instances[instances["STATUS"] == "RUNNING"]
        
    instances_names = instances_names["NAME"].values.tolist()    
    instances_names = {index + 1 : instance_name for index, instance_name in enumerate(instances_names)}

    instance_name = None
    while True:
        if len(instances_names) > 0:       
            output = clear()
            print("please choose one of the options below")

            for index in instances_names:
                print(index, instances_names[index])

            try:
                choose_option = int(input("enter your choice : "))
            except:
                print("please choose a number from the above options")
                continue

            if choose_option in instances_names.keys():
                instance_name = instances_names[choose_option]
                break
            else:
                print("please choose a number from the above options")
                continue
        else:
            break

    return instance_name

In [ ]:
choose_instance("stop")

In [ ]:
instances_state = {1 : "start", 2 : "stop"}

In [ ]:
while True:
    output = clear()
    print("please choose one of the options below")
    
    for index in instances_state:
        print(index, instances_state[index])
        
    try:
        choose_option = int(input("enter your choice : "))
    except:
        print("please choose a number from the above options")
        continue
        
    if choose_option in instances_state.keys():
        instance_state = instances_state[choose_option]
        instance_name = choose_instance(instance_state)
        
        if instance_name:
            output = process_instance(instance_state, instance_name)
            print("process completed, thank you!")
            break
        else:
            print(f'sorry no instances to {instance_state}')
            break
        break
    else:
        print("please choose a number from the above options")
        continue

In [ ]:
instances

In [ ]:
def process_instance(instance_state, instance_name):
    zone = instances[instances["NAME"] == instance_name]["ZONE"].values.tolist()[0]
    
    if instance_state in ["start", "stop"]:    
        cmd = f'gcloud compute instances {instance_state} {instance_name} --zone {zone}'
    else:
        cmd = f'gcloud compute ssh {instance_name} --zone {zone} -- -L 8080:localhost:8080'
        
    output = subprocess.check_output(cmd, shell=True)

In [ ]:
process_instance("console", "fastai-micro-vm")

In [ ]:
instances_state = {1 : "start", 2 : "stop", 3 : "console"}

In [ ]:
while True:
    output = clear()
    print("please choose one of the options below")
    
    for index in instances_state:
        print(index, instances_state[index])
        
    try:
        choose_option = int(input("enter your choice : "))
    except:
        print("please choose a number from the above options")
        continue
        
    if choose_option in instances_state.keys():
        instance_state = instances_state[choose_option]
        instance_name = choose_instance(instance_state)
        
        if instance_name:
            if choose_option == 1:
                output = process_instance(instance_state, instance_name)                
                # TODO : prompt for taking console
            elif choose_option == 2:
                output = process_instance(instance_state, instance_name)
            else:
                output = process_instance(instance_state, instance_name)
            print("process completed, thank you!")
        else:
            print(f'sorry no instances to {instance_state}')
            break
    else:
        print("please choose a number from the above options")
        continue